In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torchvision.utils import save_image
from torchvision.io import read_image
from torch.utils.data import DataLoader,Dataset, ConcatDataset, Subset
# import torchsummary
import torchattacks
from autoattack import AutoAttack
import timm
 
import time
import os
import copy
import robustbench
from robustbench.data import load_cifar10, load_imagenet
from robustbench.utils import load_model, clean_accuracy
from scripts.custom_Dataset import CustomDataset


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
from tqdm import tqdm


def calculate_accuracy(model, dataloader):
    total_samples = 0
    correct_predictions = 0
    model.eval()
    with torch.no_grad():
        for images, labels in tqdm(dataloader):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, dim=1)

            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    accuracy = correct_predictions / total_samples
    return accuracy

In [3]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
transform = transforms.Compose([transforms.ToTensor(),])
                                # transforms.Resize((224,224))])
                                # transforms.Resize((96,96)),
                    #   normalize])
invTrans = transforms.Compose([ transforms.Normalize(mean = [ 0., 0., 0. ],
                                                     std = [ 1/0.229, 1/0.224, 1/0.225 ]),
                                transforms.Normalize(mean = [ -0.485, -0.456, -0.406 ],
                                                     std = [ 1., 1., 1. ]),
                               ])
batch_size = 512
num_classes = 10

In [ ]:

# subset_indices = list(range(1500))
# batch_size = 512
# # image_folder = "../BLIP/experiment_1/stl_10/clean_test_images/"
# image_folder_1 = "../BLIP/experiment_1/stl_10/clean_train_images/"
# # label_file = '../BLIP/experiment_1/stl_10/clean_test_labels.txt'
# label_file_1 = "../BLIP/experiment_1/stl_10/clean_train_labels.txt"

# image_folder_2 = "../BLIP/experiment_1/stl_10/clean_train_generated/resized/"
# # label_file = '../BLIP/experiment_1/stl_10/clean_test_labels.txt'
# label_file_2 = "../BLIP/experiment_1/stl_10/clean_train_labels.txt"
# image_folder_3 = "../BLIP/experiment_1/stl_10/adv_test_generated_e2e/samples/"
# label_file_3 = "../BLIP/experiment_1/stl_10/adv_test_labels.txt"

# train_dataset_1 = CustomDataset(image_folder_1, label_file_1, transform=transform)
# train_dataset_2 = CustomDataset(image_folder_2, label_file_2, transform=transform)
# combined_dataset = ConcatDataset([train_dataset_1, train_dataset_2])
# train_loader = torch.utils.data.DataLoader(combined_dataset, batch_size=batch_size,
#                                      shuffle=True)
# test_dataset = CustomDataset(image_folder_3, label_file_3, transform=transform)
# subset = Subset(test_dataset, subset_indices)
# test_loader = torch.utils.data.DataLoader(subset, batch_size=batch_size,
#                                      shuffle=True)

In [24]:
image_folder = '../BLIP/experiment_1/cifar10/adv_test_generated_aa/'
label_file = '../BLIP/experiment_1/cifar10/clean_test_labels.txt'
# image_folder = '../BLIP/experiment_1/imagenet/clean_test_2048/'
# label_file = '../BLIP/experiment_1/imagenet/clean_test_labels_2048.txt'
# image_folder = '../BLIP/experiment_1/cifar10/clean_test_generated/'
# label_file = '../BLIP/experiment_1/cifar10/clean_test_labels.txt'
# # captions_file = "../BLIP/experiment_1/stl_10/clean_test_captions.txt"
test_dataset = CustomDataset(image_folder, label_file, transform=transform)

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=8)

In [9]:
# model = models.resnet50(pretrained=True)
# model = load_model(model_name='Standard', dataset='cifar10', threat_model='Linf').eval()
# model = load_model(model_name='Standard_R50', dataset='imagenet', threat_model='Linf')
# model = models.resnet50(pretrained=True).eval()
# model = models.wide_resnet50_2(pretrained=True).eval()
model = torch.load("wrn28_10_cifar10_trained_on_diffusion_images_only.pth").module.eval()
# model = timm.create_model('wide_resnet50_2', pretrained=True).eval()
model = nn.DataParallel(model).to(device)

In [10]:

images_list = []
labels_list = []

for images, labels in test_loader:
    images_list.append(images)
    labels_list.append(labels)

# Concatenate all batches
x_test = torch.cat(images_list, 0)
y_test = torch.cat(labels_list, 0)

In [11]:
acc = clean_accuracy(model, x_test, y_test, device=device, batch_size=512)
print(acc)

0.1777


In [ ]:
len(y_test)

In [25]:
i = iter(test_loader)
x,y = next(i)

# mean = [0.48145466, 0.4578275, 0.40821073]
# std = [0.26862954, 0.26130258, 0.27577711]
# def inverse_normalize(batch_tensor, mean=mean, std=std):
#     for tensor, m, s in zip(batch_tensor, mean, std):
#         tensor.mul_(s).add_(m)
#     return batch_tensor
# blip_normalize = transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
# transform_blip = transforms.Compose([transforms.Resize(size=(384,384)),
#                                 blip_normalize
#                                 ])
# resize = transforms.Compose([transforms.Resize(size=(32,32))])
# init_image = transform_blip(x)
# init_image = inverse_normalize(init_image)
# init_image = resize(init_image)

In [ ]:
# test_folder = '../BLIP/experiment_1/cifar10/test'
# count = 0
# for a_i in x_test.cpu():
#     save_image(a_i, "../BLIP/experiment_1/imagenet/clean_test_2048/{}.png".format(count))
#     count +=1

In [ ]:

imagenet_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()
    ])

In [ ]:
test_dataset = torchvision.datasets.ImageNet(root="../data/", split="val", transform=imagenet_transform)
test_loader = DataLoader(test_dataset, batch_size=1024, num_workers=8)

In [ ]:
len(np.unique(test_loader.targets))

In [ ]:
d = load_imagenet(5000, data_dir="../data/")

In [ ]:
x_test, y_test = d[0].cuda(), d[1].cuda()

In [ ]:
inputFile = "../BLIP/experiment_1/imagenet/clean_test_labels_2048.txt"

# Opening the given file in write mode
with open(inputFile, 'w') as filedata:

   # Traverse in each element of the input list 
   for item in y_test.cpu():
   
      # Writing each element of the list into the file
      # Here “%s\n” % syntax is used to move to the next line after adding an item to the file.
        filedata.write("%s\n" % item.numpy().item())

In [ ]:
y_test

In [7]:
calculate_accuracy(model, test_loader)

100%|██████████| 4/4 [00:04<00:00,  1.05s/it]


0.767578125

In [ ]:
import timm
model = timm.create_model('wide_resnet50_2', pretrained=True).cuda().eval()
# model.eval()

In [ ]:
adversary = AutoAttack(model, norm='Linf', eps=8/255, device=device)

In [ ]:
adversary.apgd.n_restarts

In [26]:
_, predicted = torch.max(model(x.cuda()), dim=1)

In [27]:
predicted[24]

tensor(5, device='cuda:0')

In [ ]:
""